# Day 10: 分布式训练与推理 (Distributed Training & Inference)

随着模型规模和数据量的增长，单张 GPU 往往无法满足训练需求。本章将介绍 PyTorch 中的并行训练机制，重点讲解 **DataParallel (DP)** 和 **DistributedDataParallel (DDP)** 的区别与使用。

**学习目标：**
- 理解模型并行 (Model Parallelism) 与数据并行 (Data Parallelism) 的区别
- 掌握 `nn.DataParallel` 的使用（简单，适用于单机多卡）
- 理解 `DistributedDataParallel` (DDP) 的核心概念（高效，适用于多机多卡）
- 了解 Hugging Face `Accelerate` 库如何简化分布式训练

**注意：**
由于 Jupyter Notebook 的交互式特性，直接演示多进程的 DDP 比较困难（通常需要通过 `torchrun` 命令行启动）。本 Notebook 主要演示 `DataParallel`，并提供 DDP 的代码模板供脚本运行使用。

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import time

## 1. 检查 GPU 数量

首先检查当前环境可用的 GPU 数量。

In [2]:
if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    print(f"可用 GPU 数量: {device_count}")
    for i in range(device_count):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("当前环境无 GPU，将使用 CPU 演示（无法体现并行加速）。")
    device_count = 0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

可用 GPU 数量: 8
GPU 0: NVIDIA GeForce RTX 2080 Ti
GPU 1: NVIDIA GeForce RTX 2080 Ti
GPU 2: NVIDIA GeForce RTX 2080 Ti
GPU 3: NVIDIA GeForce RTX 2080 Ti
GPU 4: NVIDIA GeForce RTX 2080 Ti
GPU 5: NVIDIA GeForce RTX 2080 Ti
GPU 6: NVIDIA GeForce RTX 2080 Ti
GPU 7: NVIDIA GeForce RTX 2080 Ti


## 2. DataParallel (DP)

`nn.DataParallel` 是 PyTorch 中最简单的并行方式。它通过在 batch 维度上将输入数据切分到多个 GPU 上，在每个 GPU 上复制模型副本进行前向计算，然后在主 GPU 上收集输出并计算损失，最后将梯度同步回各个 GPU。

**优点：**
- 代码修改极少，只需一行代码封装模型。

**缺点：**
- **单进程多线程**：受 Python GIL 限制，扩展性差。
- **负载不均**：主 GPU (GPU 0) 负责收集输出和计算 Loss，显存占用和计算压力通常比其他 GPU 大。
- 仅支持单机多卡。

### 2.1 定义一个简单的模型

In [3]:
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(32 * 32 * 3, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        return x

### 2.2 使用 DataParallel 封装模型

核心代码：`model = nn.DataParallel(model)`

In [4]:
model = SimpleModel()

if device_count > 1:
    print(f"使用 {device_count} 个 GPU 进行训练！")
    # 这一行是关键
    model = nn.DataParallel(model)

model.to(device)

使用 8 个 GPU 进行训练！


DataParallel(
  (module): SimpleModel(
    (fc1): Linear(in_features=3072, out_features=512, bias=True)
    (fc2): Linear(in_features=512, out_features=10, bias=True)
  )
)

### 2.3 模拟训练过程

训练循环本身几乎不需要修改。注意 `inputs` 和 `labels` 需要移动到 `device`（通常是 GPU 0），`DataParallel` 会自动将数据切分分发到其他 GPU。

In [5]:
# 准备虚拟数据
inputs = torch.randn(64, 3, 32, 32)
labels = torch.randint(0, 10, (64,))

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

print("开始模拟训练步骤...")
start_time = time.time()

for i in range(10):
    inputs, labels = inputs.to(device), labels.to(device)
    
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    
    print(f"Step {i+1}, Loss: {loss.item():.4f}")

print(f"训练完成，耗时: {time.time() - start_time:.4f}s")

开始模拟训练步骤...


/home/cseadmin/zsh/.venv/lib/python3.10/site-packages/torch/nn/modules/linear.py:134: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at /pytorch/aten/src/ATen/cuda/CublasHandlePool.cpp:270.)
  return F.linear(input, self.weight, self.bias)


Step 1, Loss: 2.3217
Step 2, Loss: 2.2402
Step 3, Loss: 2.1612
Step 4, Loss: 2.0847
Step 5, Loss: 2.0105
Step 6, Loss: 1.9387
Step 7, Loss: 1.8690
Step 8, Loss: 1.8014
Step 9, Loss: 1.7359
Step 10, Loss: 1.6724
训练完成，耗时: 3.1450s


## 3. DistributedDataParallel (DDP)

`nn.parallel.DistributedDataParallel` 是 PyTorch 推荐的并行方式，即使是单机多卡也推荐使用。

**原理：**
- **多进程**：每个 GPU 对应一个独立的进程，避免了 GIL 限制。
- **梯度同步**：每个进程拥有完整的模型副本，只处理部分数据。在反向传播时，通过 `Ring AllReduce` 算法高效同步梯度。

**核心概念：**
- **Rank**：进程的全局 ID（例如 4 张卡，Rank 为 0-3）。
- **World Size**：总进程数（总 GPU 数）。
- **Process Group**：进程组，用于通信。
- **DistributedSampler**：确保每个进程读取不同的数据切片。

### 3.1 DDP 代码结构示例 (伪代码)

DDP 通常需要通过脚本运行，以下是标准结构：

```python
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data.distributed import DistributedSampler

def setup(rank, world_size):
    # 初始化进程组
    dist.init_process_group("nccl", rank=rank, world_size=world_size)

def cleanup():
    dist.destroy_process_group()

def main(rank, world_size):
    setup(rank, world_size)
    
    # 1. 模型移动到对应 GPU
    model = SimpleModel().to(rank)
    # 2. 封装 DDP
    model = DDP(model, device_ids=[rank])
    
    # 3. 数据采样器
    dataset = ...
    sampler = DistributedSampler(dataset, num_replicas=world_size, rank=rank)
    dataloader = DataLoader(dataset, batch_size=32, sampler=sampler)
    
    # 4. 训练循环
    for epoch in range(epochs):
        sampler.set_epoch(epoch) # 重要：打乱数据
        for data, target in dataloader:
            data, target = data.to(rank), target.to(rank)
            ...
            
    cleanup()

# 启动方式：
# torchrun --nproc_per_node=4 script.py
```

## 4. Hugging Face Accelerate

手动编写 DDP 代码比较繁琐（涉及 setup, cleanup, rank 管理等）。Hugging Face 推出的 `Accelerate` 库极大地简化了这一过程，让你用同一套代码在 CPU、单 GPU、多 GPU (DDP)、TPU 上运行。

### 4.1 使用 Accelerate 改造代码

```python
from accelerate import Accelerator

accelerator = Accelerator()

model = SimpleModel()
optimizer = optim.SGD(model.parameters(), lr=0.01)
dataloader = DataLoader(...)

# 自动准备所有对象
model, optimizer, dataloader = accelerator.prepare(model, optimizer, dataloader)

for data, target in dataloader:
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    
    # 替代 loss.backward()
    accelerator.backward(loss)
    
    optimizer.step()
```

**启动方式：**
```bash
accelerate config  # 配置环境（只需一次）
accelerate launch script.py # 启动脚本
```

## 5. 总结

| 特性 | DataParallel (DP) | DistributedDataParallel (DDP) |
| :--- | :--- | :--- |
| **适用场景** | 单机多卡，快速验证 | 单机多卡，多机多卡，生产环境 |
| **实现原理** | 单进程多线程，参数服务器模式 | 多进程，Ring AllReduce |
| **性能** | 较差（GIL 锁，GPU 0 负载高） | 优（无 GIL，负载均衡） |
| **易用性** | 极简（一行代码） | 较复杂（需处理进程组、采样器） |

**建议：**
- 简单的实验或 Debug：使用 `DataParallel`。
- 正式训练或大规模数据：使用 `DDP` 或 `Accelerate`。